In [24]:
import os
import openpyxl
import cv2
import time
import datetime
from tqdm.notebook import tqdm

## 配置

In [46]:
#FairMOT算法输出的outpu.txt路径
output_txt_path = 'output/20211012/D02_20211012075959/mot_results/D02_20211012075959.txt' 
#原视频路径
source_video_path = '/home/swjtu/dataset/20211012/D02_20211012075959.mp4' 
#裁剪图片保存目录
save_crops_path = './output/20211012/D02_20211012075959/crops'   
#excel保存目录                                           
save_excel_path = "./output/20211012/D02_20211012075959/D02_20211012075959.xlsx" 
     
#原视频相机序号
source_video_camera_id = 1                                               
#原视频片段序号(按文件名升序从1开始排列)
source_video_seq_id = 1                                                     
#原视频真实起始日期时间
source_video_start_time = "20211012080011"   
                                      
#原视频帧率
source_video_frame_rate = 25.01     
#MOT算法检测帧率                                               
detection_frame_rate = 25.01        
#crop同一个ID的2张图片最小帧间隔                                               
crop_frame_interval = 10  
  

#忽略行人ID                                                       
ignore_pids = []
#阈值过滤
min_threshold = -1                                                                    
#最小行人框大小（在FairMOT中设置即可）
min_box_area = 20000                             

## 加载MOT输出结果

In [45]:
output = []
with open(output_txt_path,'r') as f:
    for line in f.readlines():
        frame, pid, x, y, w, h, threshold = line.split(',')[:7]
        frame, pid, x, y, w, h, threshold = int(frame), int(pid), float(x), float(y), float(w), float(h), float(threshold)
        if pid in ignore_pids or (min_box_area!=-1 and w*h<min_box_area) or (min_threshold!=-1 and threshold < min_threshold):
            continue
        output.append([frame, pid, x, y, w, h, threshold])
print(len(output))

4018


## 根据Output打开原视频截取行人图片

In [36]:
cap = cv2.VideoCapture(source_video_path)
cur_frame = 1
frame_ratio = source_video_frame_rate / detection_frame_rate
index = 0
len_output = len(output)
if not os.path.exists(save_crops_path):
    os.makedirs(save_crops_path)
pbar = tqdm(total=len_output)
pbar.set_description('Processing:')
last_crop_frame_dic = {}
for i in output:
    # 初始化为-crop_frame_interval
    last_crop_frame_dic[i[1]] = -crop_frame_interval 
while index < len_output:
    ret, img = cap.read()   
    if ret == False or img is None: break
    while index < len_output and cur_frame == output[index][0]:
        frame, pid, x, y, w, h, threshold = output[index]
        index += 1
        pbar.update(1)
        # if (cur_frame-1) % crop_frame_interval != 0:
        # 当前帧的pid距离上一次crop小于crop_frame_interval帧则跳过
        if cur_frame - last_crop_frame_dic[pid] < crop_frame_interval:
            continue
        last_crop_frame_dic[pid] = cur_frame
        #通过帧号计算日期时间
        time = datetime.datetime.strptime(source_video_start_time, "%Y%m%d%H%M%S") + datetime.timedelta(seconds=round(frame/detection_frame_rate))
        fileName = "%04d_c%02ds%02d_%04d_%s.jpg" % (pid, source_video_camera_id, source_video_seq_id, round(frame * frame_ratio)
                                                    , time.strftime("%Y%m%d%H%M%S"))
        if x*y*w*h > 0:
            cv2.imwrite(os.path.join(save_crops_path, fileName), img[round(y):round(y+h), round(x):round(x+w)])
        
    cur_frame += 1

  0%|          | 0/4018 [00:00<?, ?it/s]

## 创建Excel并写入数据

In [33]:
wb = openpyxl.Workbook()
ws = wb['Sheet']
ws.append(['人员ID', '进入时间', '离开时间', '进入帧号', '离开帧号', '摄像头ID', '视频片段序号'])

idMap = {}
for frame,pid,x1,y1,w,h,threshold in output:
    if pid not in idMap:
        idMap[pid] = [frame, frame] # in_frame, out_frame
    else:
        idMap[pid][1] = frame
for pid in idMap:
    # 通过帧号计算视频内的时间轴信息
    in_time, out_time = [ "%02d:%02d:%02d" % (time//3600, time%3600//60, time%60) for time in [ frame // detection_frame_rate for frame in idMap[pid]] ]
    ws.append([pid, in_time, out_time, idMap[pid][0], idMap[pid][1], source_video_camera_id, source_video_seq_id])
wb.save(save_excel_path)